In [2]:
from cvxopt import matrix, solvers
import numpy as np
import random

In [21]:
def bp(X, y):
    '''
    (X,y): Matriz de datos y vector de respuesta [X.shape=(m,p), y.shape=(m,)]
    (betahat): Solución [betahat.shape=(1,)]
    '''
    
    m=matrix(X.shape[0])
    p=matrix(X.shape[1])
    c=matrix(np.ones(2*p[0]))
    A=matrix(np.append(X,-X, axis=1))
    b=matrix(y)
    G=matrix(-np.identity(2*p[0]))
    h=matrix(np.zeros(2*p[0]))
    
    sol = solvers.conelp(c, G, h,A=A,b=b)
    res = sol['x']
    res = np.array(res).flatten()
    betamas,betamenos=np.split(res,2)
    betahat = betamas-betamenos
    
    return betahat

In [22]:
def datos(m,p,k):
    np.random.seed(1111)
    random.seed(1111)
    X = np.random.normal(0,1,(m,p))
    beta = np.random.normal(0,1,(p,1))
    beta[random.sample(range(0,p), k=p-k)]=0
    y=np.matmul(X,beta)
    return X,y.squeeze(), beta.squeeze()

In [23]:
X,y,beta=datos(75,150,5)
betahat=bp(X,y)

     pcost       dcost       gap    pres   dres   k/t
 0: -5.5511e-17 -1.7237e-16  4e+02  2e+01  2e-16  1e+00
 1:  6.9780e-01  6.9907e-01  6e+01  3e+00  2e-16  1e-01
 2:  2.7180e+00  2.7198e+00  1e+01  9e-01  7e-16  4e-02
 3:  2.8037e+00  2.8062e+00  4e+00  3e-01  5e-16  1e-02
 4:  3.1002e+00  3.1003e+00  9e-02  5e-03  6e-16  3e-04
 5:  3.1036e+00  3.1036e+00  9e-04  5e-05  3e-16  3e-06
 6:  3.1036e+00  3.1036e+00  9e-06  5e-07  3e-16  3e-08
 7:  3.1036e+00  3.1036e+00  9e-08  5e-09  3e-16  3e-10
Optimal solution found.


In [24]:
np.linalg.norm(betahat-beta)

2.050021836322254e-09

In [25]:
print(f'''Soporte de beta: [{str(min(beta))}, {str(max(beta))}]
Soporte de betahat: [{str(min(betahat))}, {str(max(betahat))}]''')

Soporte de beta: [-0.9956062679340661, 0.8167211788085769]
Soporte de betahat: [-0.9956062674257806, 0.8167211781936211]


Supongamos un conjunto de datos con $p\gg m$ al que se quiere ajustar un modelo lineal. Para encontrar la solución más simple dentro de la infinidad posible, elegimos una que tenga el menor número de entradas no cero. Es decir, si definimos

\begin{equation} |\beta|_0 = #{j\in [p] : \beta_j\neq 0} \end{equation}

elegimos la beta que minimice $|\beta|_0$ y satisfaga $X'\beta=y$.

Primero, notar que $|\beta|_q \rightarrow |\beta|_0$ cuando $q\rightarrow 0^+$, pues las entradas que ya son cero lo siguen siendo sin importar el exponente y las que no son cero se van a uno cuando $q\rightarrow 0^+$. El resultado se sigue sumando límites.
Sin embargo, no es una norma porque por ejemplo, $|(1, 0)|_0=1$, $|(-1, 0)|=1$ y la norma cero de su suma es cero.

Se necesitarían $2^p$ evaluaciones para hacerlo por fuerza bruta incluso sin considerar el error.

Mostraremos que el problema es NP duro reduciendo el problema de cubiertas exactas de 3-conjuntos a uno de esta forma en tiempo polinomial.

Dados los conjuntos $S_j$, construimos una matriz $X\in M_{m\times p}(\mathbb{R})$ con columnas $(x_1 \mid \cdots \mid x_p)$ donde $(x_i)_j = 1$ si y sólo si $j \in S_i$. Esta construcción es $O(mp)$.

Sea $y \in \mathbb{R}^m$ el vector de unos. Si resolvemos el problema de minimizar $(1)$, obtendremos un vector $\beta$ con muchas entradas cero. Al multiplicar por nuestra matriz $X$, las columnas donde $\beta$ sea cero se van a cancelar.